In [16]:
import json 
import requests
import time
import urllib.request

TOKEN = "865979318:AAGfil3jAjgM8Qzl9m_j6uvksmPC7U7RkM8"
URL = "https://api.telegram.org/bot{}/".format(TOKEN)
URL_FILE = "https://api.telegram.org/file/bot{}/".format(TOKEN)


def get_url(url):
    response = requests.get(url)
    content = response.content.decode("utf8")
    return content


def get_json_from_url(url):
    content = get_url(url)
    js = json.loads(content)
    return js


def get_updates():
    url = URL + "getUpdates"
    js = get_json_from_url(url)
    return js


def get_last_chat_id_and_text(updates):
    num_updates = len(updates["result"])
    last_update = num_updates - 1
    global tipekonten
    try:
        konten = updates["result"][last_update]["message"]["photo"][2]["file_id"]
        tipekonten="gambar"
    except KeyError:
        try:
            konten = updates["result"][last_update]["message"]["text"]
            tipekonten="text"
        except KeyError:
            True
    
    chat_id = updates["result"][last_update]["message"]["chat"]["id"]
    
    return ( tipekonten,konten, chat_id)


def send_message(text, chat_id):
    url = URL + "sendMessage?text={}&chat_id={}".format(text, chat_id)
    get_url(url)
def send_file(file_id, chat_id):
    url = URL + "sendPhoto?photo={}&chat_id={}".format(file_id, chat_id)
    get_url(url)
def save_file(file_id, chat_id):
    url = URL + "getFile?file_id={}".format(file_id)
    js = get_json_from_url(url)
    file_path = js["result"]["file_path"]
    url = URL_FILE + file_path
    testfile = urllib.request.urlretrieve(url,+"/"+file_id+".jpg")
    send_message("image downloaded",chat_id)
    
def prosedur():
    
    
def main():
    last_textchat = (None, None)
    while True:
        tipekonten, konten, chat = get_last_chat_id_and_text(get_updates())
        if (konten, chat) != last_textchat:
            if(tipekonten=="text"):
                send_message(konten, chat)
            elif(tipekonten=="gambar"):
                save_file(konten, chat)
            last_textchat = (konten, chat)
        time.sleep(0.5)


if __name__ == '__main__':
    main()


From cffi callback <function _verify_callback at 0x0000021B38BB0620>:
Traceback (most recent call last):
  File "C:\Users\Ditzy\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt


SSLError: HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot865979318:AAGfil3jAjgM8Qzl9m_j6uvksmPC7U7RkM8/getUpdates (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))